<a href="https://colab.research.google.com/github/naafey-aamer/xray_classifier/blob/main/Pneumonia_Classifier_EffiecientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install efficientnet
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
import efficientnet.tfkeras as efn

# Load the pre-trained ResNet-50 model (without the top fully connected layers)
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# replacing ResNet50 with EfficientNetB0
base_model = efn.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(180, 180, 3))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define data directories for train, validation, and test sets
train_data_dir = '/content/drive/MyDrive/xray_pneu/train'
val_data_dir = '/content/drive/MyDrive/xray_pneu/val'
test_data_dir = '/content/drive/MyDrive/xray_pneu/test'

# Set up data generators for train, validation, and test sets
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(180, 180),
    batch_size=128,
    class_mode='binary',
    classes=['NORMAL', 'PNEUMONIA']  # Classes based on subfolder names
)

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(180, 180),
    batch_size=4,
    class_mode='binary',
    classes=['NORMAL', 'PNEUMONIA']
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(180, 180),
    batch_size=32,
    class_mode='binary',
    classes=['NORMAL', 'PNEUMONIA']
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
# print(train_generator[])

In [ ]:
# Freeze pre-trained layers
# for layer in base_model.layers:
#     layer.trainable = False

# Unfreeze all layers
for layer in base_model.layers:
    layer.trainable = True

# print(len(base_model.layers))

# keep the bottom layers (first 170) frozen
for layer in base_model.layers[:220]:
    layer.trainable = False

# new dense layers for binary classification
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dropout(0.65)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.65)(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# new model with the added layers
model = tf.keras.Model(inputs=base_model.input, outputs=output)
model.compile(
    loss='binary_crossentropy',
    optimizer=SGD(lr=0.001, momentum=0.9),
    metrics=['accuracy']
)

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 180, 180, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv (Conv2D)             (None, 90, 90, 32)   864         ['input_4[0][0]']                
                                                                                                  
 stem_bn (BatchNormalization)   (None, 90, 90, 32)   128         ['stem_conv[0][0]']              
                                                                                                  
 stem_activation (Activation)   (None, 90, 90, 32)   0           ['stem_bn[0][0]']          

In [ ]:
def lr_schedule(epoch, lr):
    # Reduce learning rate by 20% every 2 epochs
    if (epoch+1) % 2 == 0:
        lr *= 0.8
    return max(lr, 3e-4)

# Calculate class weights to handle the class imbalance
total_train_samples = 1342 + 3876
class_weights = {
    0: total_train_samples / (2 * 1342),   # Weight for 'NORMAL' class
    1: total_train_samples / (2 * 3876)    # Weight for 'PNEUMONIA' class
}

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule, verbose=1)
earlystop = EarlyStopping(monitor='val_loss', patience=2, verbose=1)

# Fit the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=14,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=[lr_callback, earlystop],
    class_weight=class_weights
)


Epoch 1: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 1/14
41/41 [==============================] - 402s 9s/step - loss: 0.4179 - accuracy: 0.7866 - val_loss: 0.3746 - val_accuracy: 0.8125 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.007999999821186066.
Epoch 2/14
41/41 [==============================] - 369s 9s/step - loss: 0.2171 - accuracy: 0.9114 - val_loss: 0.2797 - val_accuracy: 0.9375 - lr: 0.0080

Epoch 3: LearningRateScheduler setting learning rate to 0.007999999448657036.
Epoch 3/14
41/41 [==============================] - 374s 9s/step - loss: 0.1781 - accuracy: 0.9233 - val_loss: 0.2629 - val_accuracy: 0.8750 - lr: 0.0080

Epoch 4: LearningRateScheduler setting learning rate to 0.006399999558925629.
Epoch 4/14
41/41 [==============================] - 373s 9s/step - loss: 0.1621 - accuracy: 0.9354 - val_loss: 0.2222 - val_accuracy: 0.9375 - lr: 0.0064

Epoch 5: LearningRateScheduler setting learning rate to 0.006399999

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f'Test accuracy: {test_accuracy}')

20/20 [==============================] - 32s 2s/step - loss: 0.3898 - accuracy: 0.8750
Test accuracy: 0.875
